In [6]:
# dependencies
%matplotlib inline
%load_ext autoreload
%autoreload 2

import os
import h5py
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable
from matplotlib import cm 
import scipy.stats as st
from scipy.stats import zscore
import numpy.random as npr
import ssm
import seaborn as sns
from ssm.util import one_hot, find_permutation
from ssm.plots import gradient_cmap, white_to_color_cmap
from datetime import date

from preProcessing import * #my module
# from plottingfunctions import * #my module

#to add to filenames as datestring
today = date.today()
datestring = today.strftime("%m%d%y")

npr.seed(0)

sns.set_style("white")
sns.set_context("talk")

color_names = [
    "windows blue",
    "red",
    "amber",
    "faded green",
    "dusty purple",
    "orange",
    "maroon",
    "slate",
    "light violet",
    "golden yellow",
    "bright pink",
    "bright turquoise"
    ]

genotype_colors = sns.xkcd_palette(color_names)

colors = sns.xkcd_palette(color_names)
cmap = gradient_cmap(colors)

# sns.set(palette="colorblind")
# default_colors = sns.color_palette("colorblind")

sns.set(palette="bright")
default_colors = sns.color_palette("bright")


# Speficy whether or not to save figures
save_figures = True

#load style params
#locate data directory
dataDir = '/Users/eliasscheer/Dropbox/ROCKEFELLER/BARGMANN_LAB_LYFE/LAWNLEAVINGPROCESSEDDATA/ssm/notebooks'
figpath = '/Users/eliasscheer/Dropbox/ROCKEFELLER/THESIS/Chapter2/figures/completed_v1'
plt.style.use(os.path.join(dataDir,'../styleparams/presentation.mplstyle'))

#used for all new data loaded in
binSize = 30
minFillLen = binSize

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [7]:
#Load PD1074 HMM models and scalers
import pickle

#change to the correct path
processDataPath = '/Users/eliasscheer/Dropbox/ROCKEFELLER/BARGMANN_LAB_LYFE/LAWNLEAVINGPROCESSEDDATA/MATFILEDATA/Pickles'
with open(os.path.join(processDataPath,'PD1074_od2_LL_Data_RoamingDwellingHMM_062321.pkl'),'rb') as f:  # Python 3: open(..., 'rb')
    RD_hmm_exog = pickle.load(f)
    
#Load PD1074 scalers
with open(os.path.join(processDataPath,'PD1074_scalers_062321.pkl'),'rb') as f:  # Python 3: open(..., 'rb')
    Scalers = pickle.load(f)
    
#load HMM Models
with open(os.path.join(processDataPath,'HMM_OnLawnOnly_ForwardFeaturesOnly_noQuirk_AR_062321.pkl'),'rb') as f:
    obs_dim,N_iters,num_states,kappa,arHMMs_ALL, arHMMs_logLike_ALL,\
                 Obs_train_forHMM, Obs_test_forHMM, \
                 Choice_train_forHMM, Choice_test_forHMM = pickle.load(f)

arHMM_model = arHMMs_ALL[4][2] #PD1074 K=4, KAPPA = 25,000


In [8]:
#load PD1074 od2 LL centroid RD_hmm to be used as exog
import pickle

processDataPath = '/Users/eliasscheer/Dropbox/ROCKEFELLER/BARGMANN_LAB_LYFE/LAWNLEAVINGPROCESSEDDATA/MATFILEDATA/Pickles'
 
with open(os.path.join(processDataPath,'PD1074_od2_LL_Data_Centroid_RoamingDwellingHMM_081721.pkl'),'rb') as f:  # Python 3: open(..., 'rb')
    RD_hmm_Cent_exog = pickle.load(f)


In [22]:
#Make a dictionary to hold all data
DataList = dict()

In [90]:
#Small Lawns
import glob
import re

#wherever post-tracked analysis files are
dataPath = '/Users/eliasscheer/Dropbox/ROCKEFELLER/BARGMANN_LAB_LYFE/LAWNLEAVINGPROCESSEDDATA/MATFILEDATA/current_analysis/smallLawns/'
os.chdir(dataPath)
FeatureFiles = glob.glob('*newFeatures.mat')
matchingFileNames = [re.split("newFeatures.mat",mat)[0]+'Filenames.h5' for mat in FeatureFiles]
identifier = [re.split("\d{2}_\d{2}_\d{2}_",fn)[0]+"Data" for fn in matchingFileNames]

for mat, fn, ide in zip(FeatureFiles, matchingFileNames, identifier):
    bins, Data = preprocessData(dataPath, mat, fn, binSize, minFillLen, RD_hmm_exog, RD_hmm_Cent_exog, Scalers, arHMM_model,dataType='LL')
    DataList[ide] = Data

Processing: ZD653_OLLkill_od2_09_13_21_newFeatures.m ...
Data type is LL
Videoname size = (35, 1)
Read in lawn-related data...
Read in features and fill missing data for sufficiently short intervals...


/Users/eliasscheer/PycharmProjects/lawnleaving/preProcessing.py:643: RuntimeWarning: invalid value encountered in double_scalars
  dotpdt = np.dot(DirVector_v12, DirVector_v23) / (norm(DirVector_v12) * norm(DirVector_v23))


bin Data...


/Users/eliasscheer/PycharmProjects/lawnleaving/preProcessing.py:681: RuntimeWarning: Mean of empty slice
  binDataOut[vid, i] = function(currDataChunk)  # function, e.g. nanmean


  0%|          | 0/100 [00:00<?, ?it/s]

...

/Users/eliasscheer/PycharmProjects/lawnleaving/preProcessing.py:643: RuntimeWarning: invalid value encountered in double_scalars
  dotpdt = np.dot(DirVector_v12, DirVector_v23) / (norm(DirVector_v12) * norm(DirVector_v23))


bin Data...


/Users/eliasscheer/PycharmProjects/lawnleaving/preProcessing.py:681: RuntimeWarning: Mean of empty slice
  binDataOut[vid, i] = function(currDataChunk)  # function, e.g. nanmean


  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

In [13]:
#Make sub dictionaries for different experiment groups 

wt_keys = ['PD1074_od1_Data', 'PD1074_od2_Data', 'PD1074_od4_Data', 'N2_od1_Data', 'N2_od2_Data', 'N2_od5_Data','PD1074_Sep2022_od2_Data']
wildtype =  {key: DataList[key] for key in wt_keys}



In [ ]:
#save dictionaries into separate pickles
processDataPath = '/Users/eliasscheer/Dropbox/ROCKEFELLER/BARGMANN_LAB_LYFE/LAWNLEAVINGPROCESSEDDATA/MATFILEDATA/Pickles'

# with open(os.path.join(processDataPath,'WildType_022022.pkl'),'wb') as f:
#     pickle.dump([bins,binSize,wildtype],f)
# with open(os.path.join(processDataPath,'WildType_022022.pkl'),'rb') as f:  # Python 3: open(..., 'rb')
#     bins,binSize,wildtype = pickle.load(f)

